# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_copy import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
weather_csv_file = "cities_copy.csv"

weather_df = pd.read_csv(weather_csv_file)
weather_df = weather_df.drop(weather_df.columns[0], axis =1)

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,buarcos,40.17,-8.88,73.99,78.0,20.0,12.75,PT,1.595939e+09
1,jamestown,42.10,-79.24,72.00,88.0,1.0,5.82,US,1.595939e+09
2,vaini,-21.20,-175.20,69.80,77.0,20.0,10.29,TO,1.595939e+09
3,kapaa,22.08,-159.32,78.80,83.0,90.0,19.46,US,1.595939e+09
4,karpinsk,59.77,60.01,68.40,45.0,82.0,4.76,RU,1.595939e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
%matplotlib inline
#configure gmaps with apikeys
gmaps.configure(api_key=g_key)

#store lat and lng as locations
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Narrow down based on specified criterias

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,lagoa,39.05,-27.98,73.09,86.0,0.0,7.00,PT,1.595939e+09
191,ponta do sol,32.67,-17.10,75.00,64.0,0.0,1.77,PT,1.595939e+09
247,ribeira brava,32.65,-17.07,75.00,74.0,0.0,5.99,PT,1.595939e+09
342,hays,30.05,-98.03,75.99,98.0,0.0,2.93,US,1.595939e+09
366,zhangjiakou,40.81,114.88,70.97,72.0,0.0,5.55,CN,1.595939e+09
368,pingliang,35.54,106.69,70.90,60.0,0.0,3.09,CN,1.595939e+09
374,ilhabela,-23.78,-45.36,79.65,53.0,0.0,4.29,BR,1.595939e+09
403,kruisfontein,-34.00,24.73,72.00,35.0,0.0,5.99,ZA,1.595939e+09
426,shirokiy,49.76,129.52,72.59,76.0,0.0,6.22,RU,1.595939e+09
434,paciran,-6.88,112.38,78.75,74.0,0.0,4.52,ID,1.595939e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df = filtered_weather_df[["City", "Country","Lat","Lng"]]
#adding columns
hotel_df["Hotel Name"] = ""

# setting parameters
params = {
    "radius": 5000,
    "types" : "lodging",
    "key": g_key
}


/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
hotel_df

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
34,lagoa,PT,39.05,-27.98,Casa Das Faias
191,ponta do sol,PT,32.67,-17.10,Hotel do Campo
247,ribeira brava,PT,32.65,-17.07,Hotel do Campo
342,hays,US,30.05,-98.03,Sinya on Lone Man Creek
366,zhangjiakou,CN,40.81,114.88,Shell Zhangjiakou City Qiaodong District Ginza...
368,pingliang,CN,35.54,106.69,Xinxiang Hostel
374,ilhabela,BR,-23.78,-45.36,Ilha Flat Hotel
403,kruisfontein,ZA,-34.00,24.73,Oyster Bay House Rental
426,shirokiy,RU,49.76,129.52,
434,paciran,ID,-6.88,112.38,Tanjung Kodok Beach Resort


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map


# Display figure


In [36]:
marker_output= gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_output)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))